# Creation of the CNN

### Importations 

In [56]:
import torch
import matplotlib.pyplot as plt
import numpy as np
import pickle as pkl
import pandas as pd


from torch.autograd import Variable
from torch import nn, optim
from torch.nn import functional as F
from matplotlib import gridspec




print(torch.__version__)

1.0.1.post2


### The Aim 

Some explation about the purpose of this project.

### The Data

Some insight about the data.

In [57]:
Dataset = pd.read_pickle("/home/janjar/Dataset/Trainingset/Training_dataframe.pkl")
Dataset.loc[0:3]

,Room,Array_position,Distance,Absorption,SNR,Audio_file,Phase_Matrix,Label
0,0,"[3.395545610109257, 2.0550302704519794, 1.5]",1.0,0.702471,0,/home/janjar/Dataset/Trainingset/audio_signals...,/home/janjar/Dataset/Trainingset/phase_matrix/...,21
1,0,"[3.2402107165724896, 1.8276250338752629, 1.5]",1.0,0.702471,0,/home/janjar/Dataset/Trainingset/audio_signals...,/home/janjar/Dataset/Trainingset/phase_matrix/...,32
2,0,"[1.6801067878855174, 1.927483519251545, 1.5]",1.0,0.702471,0,/home/janjar/Dataset/Trainingset/audio_signals...,/home/janjar/Dataset/Trainingset/phase_matrix/...,9
3,0,"[3.2065269137273935, 3.207686173511143, 1.5]",1.0,0.702471,0,/home/janjar/Dataset/Trainingset/audio_signals...,/home/janjar/Dataset/Trainingset/phase_matrix/...,27


#### Defining our number of samples.

In [59]:
number_of_rooms = Dataset.shape[0]

### Loading the phase matrices in tensors with respect to the indexing shown in the dataframe.

In [23]:
#path containing the Training data on the machine you are working on.
#path = "/home/janjar/Dataset/Trainingset/"

data_matrix = np.zeros([number_of_rooms,4,129,390])
for i in range(number_of_rooms):
    fileName_matrix = Dataset.iloc[i]['Phase_Matrix']
    fileObject2 = open(fileName_matrix, 'rb')
    matrix_loaded = pkl.load(fileObject2)
    fileObject2.close()
    data_matrix[i] = matrix_loaded
data_matrix.shape    

(140, 4, 129, 390)

In [24]:
data_matrix = torch.from_numpy(data_matrix)
data_matrix = data_matrix.view(-1,390,4,129)
data_matrix = data_matrix.type('torch.FloatTensor')
data_matrix.shape

torch.Size([140, 390, 4, 129])

### Function that reshape a label into the right format for the data_targets.

In [32]:
def prep_targets(index,label):
    target = torch.tensor(label)
    target = target.expand(390,1)
    return target

In [42]:
data_targets = torch.zeros([number_of_rooms,390,1], dtype=torch.float64)
for i in range(number_of_rooms):
    data_targets[i] = prep_targets(i,Dataset.iloc[i]['Label'])
data_targets.shape   

torch.Size([140, 390, 1])

# Architecture 1.0 : vanilla CNN for DOA 

In this section, we will focus on building a vanilla CNN in order to recognize the directions of arrival of the sound in a specific room. Once done, we will then compare "by hand" the labelel dataset to the predicted values. This is the most basic setup and will try to improve latter.

## 1.1 Creating Model

In [43]:
class Net(nn.Module):
    def __init__(self,nb_hidden = 512):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 64, kernel_size=2)
        self.conv2 = nn.Conv2d(64, 64, kernel_size=2)
        self.conv3 = nn.Conv2d(64 , 64, kernel_size=2)
        self.fc1 = nn.Linear(8064, nb_hidden)# (1x8064) being the dim of the censor obtained by flattening the output of the 3rd CL.
        self.fc2 = nn.Linear(nb_hidden, 37)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = F.relu(self.conv3(x))
        x = F.relu(self.fc1(x.view(-1, 8064)))
        x = F.relu(self.fc2(x))
       
        return F.softmax(x)

### Preparing the Data

In [44]:
def prep_input_vanilla(train_input):
    new_train_input = train_input.view(-1,390,1,1,4,129)
    return new_train_input

In [45]:
train_data = prep_input_vanilla(data_matrix)
train_data.shape

torch.Size([140, 390, 1, 1, 4, 129])

In [47]:
def prep_labels_vanilla(train_input):
    new_train_input = train_input.view(-1,390,1)
    return new_train_input

In [64]:
train_targets = prep_labels_vanilla(data_targets)
train_targets = train_targets.to(dtype=torch.int64)
train_data, train_targets = Variable(train_data), Variable(train_targets)
train_targets.shape

torch.Size([140, 390, 1])

### End of Sanity check.

## 1.2 Training the model

In [65]:
mini_batch_size = 30 
nb_epochs = 5
eta = 0.5 #learning rate
#criterion = nn.MSELoss() # MeanSquaredloss
criterion = torch.nn.CrossEntropyLoss() #Cross Entropy

### Need to figure out how to use mini-batches to gain time in this configuration.
#### train_model is the method used to update the weights.

In [68]:
def train_model(model, train_input, train_target, mini_batch_size, criterion, nb_epochs, eta):
    
    optimizer = torch.optim.SGD(model.parameters(), lr = eta, momentum = 0.95) #Stochastic gradient descent
    
    for e in range(0, nb_epochs):
        # We do this with mini-batches
        for b in range(0, train_input.size(0)):
            for a in range(0,train_input.size(1)):
                output = model(train_input[b,a,:,:,:,:])
                target = train_target[b,a,:]
                #print("Output vs target",output,target)
                #loss = criterion(output,target)
                loss = criterion(output,target)
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()
            
        print(e, loss.data.item())
    

In [69]:
def train_model_2(model, train_input, train_target, mini_batch_size, criterion, nb_epochs, eta):
    
    optimizer = torch.optim.SGD(model.parameters(), lr = eta, momentum = 0.95) #Stochastic gradient descent
    
    for e in range(0, nb_epochs):
        # We do this with mini-batches
        for b in range(0, train_input.size(0)):
            for a in range(0,train_input.size(1),mini_batch_size):
                output = model(train_input[b].narrow(0, a, mini_batch_size))
                #print(output.shape)
                target = train_targets[b].narrow(0, a, mini_batch_size)
                #print(target.shape)
                #loss = criterion(output,target)
                loss = criterion(output,targets.max(1)[1])
                print(b,target.max(1)[1])
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()

        print(e, loss.data.item())

In [70]:
model = Net()

In [ ]:
train_model(model,train_data,train_targets,mini_batch_size,criterion,nb_epochs,eta)

/home/janjar/test/lib/python3.5/site-packages/ipykernel_launcher.py:17: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


0 3.610917806625366
